In [15]:
import pandas as pd

def complete_ages(df_train, df_test):
    # Obtener el título de la columna nombre.
    df_train['Title'] = df_train['Name'].str.extract(' ([A-Za-z]+)\\.', expand=False)

    # Obtener la edad promedio para cada título extraido 
    average_age_per_title = df_train.groupby('Title')['Age'].mean()

    # Llenar los valores que hacen falta en la columna de edad utilizando el promedio de edad dependiendo de el título utilizando tomando en cuenta la desviación estandar.
    for title in average_age_per_title.index:
        df_train.loc[(df_train['Age'].isnull()) & (df_train['Title'] == title), 'Age'] = average_age_per_title[title] + df_train['Age'].std()

    # Redondear los valores correspondientes a la edad al int más cercano.
    df_train['Age'] = df_train['Age'].round()
    
    # Repetir el mismo proceso en el dataset de test.
    df_test['Title'] = df_test['Name'].str.extract(' ([A-Za-z]+)\\.', expand=False)
    average_age_per_title_test = df_test.groupby('Title')['Age'].mean()
    for title in average_age_per_title_test.index:
        df_test.loc[(df_test['Age'].isnull()) & (df_test['Title'] == title), 'Age'] = average_age_per_title_test[title] + df_test['Age'].std()
    df_test['Age'] = df_test['Age'].round()

    # Repetir el proceso e el dataset de testeo utilizando el promedio de edades por título del dataset de entrenamiento.
    for title in average_age_per_title.index:
        df_test.loc[(df_test['Age'].isnull()) & (df_test['Title'] == title), 'Age'] = average_age_per_title[title] + df_test['Age'].std()
    df_test['Age'] = df_test['Age'].round()

    # Eliminar la columna de ¨Title¨ de ambos conjuntos de datos.
    df_train = df_train.drop(columns=['Title'])
    df_test = df_test.drop(columns=['Title'])

    return df_train, df_test  

# Load data/train.csv
train = pd.read_csv('data/train.csv')

# Load data/test.csv
test = pd.read_csv('data/test.csv')

# Complete missing ages
train, test = complete_ages(train, test)

# Load data/sample_submission.csv
gender_submission = pd.read_csv('data/gender_submission.csv')

# Merge test and sample_submission with PassengerId
test = test.merge(gender_submission, on='PassengerId')

# Combine train and test
data = pd.concat([train, test], sort=False)

# Remove "" from 'Name' column
data['Name'] = data['Name'].str.replace('"', '')

# Round age to int
data['Age'] = data['Age'].round()

# Make age column int
data['Age'] = data['Age'].astype(int)

# Only keep 'PassengerId', 'Survived', 'Age', 'Name'
data = data[['PassengerId', 'Survived', 'Age', 'Name']]

# Check nans in data
print(data.isnull().sum())

# Export data to data/all_data.csv
data.to_csv('data/all_data.csv', index=False)

# Export 

PassengerId    0
Survived       0
Age            0
Name           0
dtype: int64
